In [3]:
from collections import namedtuple
import json
import pathlib
import re
from typing import Dict, List, Optional

import clickhouse_connect
import nltk
import pandas as pd
import tqdm

In [15]:
with open("stopwords.json") as f:
    stopwords = json.load(f)

In [16]:
Owner = namedtuple("Owner", ["name", "country_code", "individual"])
Patent = namedtuple("Patent", ["number", "owners", "address"])
Person = namedtuple("Person", ["name", "tax_number"])

In [17]:
client = clickhouse_connect.get_client(
    host="213.171.29.36",
    username="phd",
    password="phd",
)
client.command("SELECT version()")

'24.5.1.1763'

In [18]:
client.command("SELECT count(*) FROM search.search_base")

28370248

In [19]:
def parse(row: pd.Series) -> Patent:
    row = row.fillna("")
    owner_names = [
        name.strip().replace("\n", "").replace("\r", "")
        for name in row["patent holders"].split("\r\n")
    ]
    address = row["correspondence address"]
    number = row["registration number"]
    
    regex = re.compile("\((?a:\w{2})\)")
    owners = []
    for name in owner_names:
        country_code = None
        individual = False
        
        country_code_match = regex.search(name)
        if country_code_match is not None:
            country_code = country_code_match.group(0)[1:-1]
            name = name.replace(country_code_match.group(0), "").strip()
        
        if row["authors"] == row["patent holders"]:
            individual = True
        
        name_parts = list(filter(lambda x: len(x) > 0, map(str.strip, name.split(" "))))
        if (
            len(name_parts) == 3
            and all(part[0].isupper() for part in name_parts)
        ):
            individual = True
        if (
            len(name_parts) == 2 
            and name_parts[0][0].isupper()
            and name_parts[1].replace(".", "").isupper()
        ):
            individual = True
        
        owners.append(Owner(name, country_code, individual))
    
    if (
        len(owners) > 1
        and all(owner.individual is False for owner in owners)
        and sum([owner.country_code is not None for owner in owners]) == 1
    ):
        country_code = [owner.country_code is not None for owner in owners][0]
        owners = [
            Owner(
                " ".join(owner.name for owner in owners),
                country_code,
                False
            )
        ]
        
    return Patent(number=number, owners=owners, address=address)

In [20]:
def preprocess_name_for_exact_match(name: str) -> str:
    name = name.upper()
    name = name.replace("ИНДИВИДУАЛЬНЫЙ ПРЕДПРИНИМАТЕЛЬ", "").strip()
    
    match = re.search("([А-Я]\.)([А-Я]\.)", name)
    if match is not None:
        name = name.replace(match[0], f"{match[1]} {match[2]}")
    
    return name


def search_by_exact_name_match(name: Optional[str], address: Optional[str], individual: bool) -> List[Person]:
    if name is None or name == "":
        return []
    
    if address is None or address == "":
        address = "фыва" # meaningless, score ~ 1 (least similar) for ngramDistance
    
    orig_name = name
    name = preprocess_name_for_exact_match(name)
    
    stmt = """
        SELECT
            name,
            tax_number,
            ngramDistance(legal_address, {address:String}) as dal,
            ngramDistance(fact_address, {address:String}) as daf,
            dal * daf as score
        FROM search.search_base
        WHERE 
            name = {name:String}
            AND individual = {individual:bool}
        ORDER BY score
        LIMIT 2
    """
    
    params = {
        "name": name, 
        "address": address,
        "individual": individual,
    }    
    
    res = client.query(stmt, parameters=params)

    if len(res.result_rows) == 0:
        return []
    else:
        return [
            Person(name=row[0], tax_number=row[1])
            for row in res.result_rows
        ]

In [21]:
search_by_exact_name_match("Индивидуальный предприниматель МЕЛЬНИЧЕНКО ИГОРЬ ЮРЬЕВИЧ", "121165 Москва а/я 15 ООО \"Юстис\" Грунина А.Е.", individual=True)

[Person(name='МЕЛЬНИЧЕНКО ИГОРЬ ЮРЬЕВИЧ', tax_number='771670514800')]

In [22]:
def unquote_name(name: str) -> str:
    name = name.replace("«", '"')
    name = name.replace("»", '"')
    
    match = re.search("([А-Я]\.)([А-Я]\.)", name)
    if match is not None:
        name = name.replace(match[0], f"{match[1]} {match[2]}")
    
    if '"' not in name:
        return name
    
    parts = name.split('"')
    if len(parts) <= 4:
        return parts[1]
    else:
        return parts[2]

def search_by_like_name_match(name: Optional[str], address: Optional[str], individual: bool) -> List[Person]:
    if name is None or name == "":
        return []
    
    if address is None or address == "":
        address = "фыва" # meaningless, score ~ 1 (least similar) for ngramDistance
    
    stmt = """
        SELECT
            name,
            tax_number,
            ngramDistance(name, {original_name:String}) as dn,
            ngramDistance(legal_address, {address:String}) as dal,
            ngramDistance(fact_address, {address:String}) as daf,
            dn * dal * daf as score
        FROM search.search_base
        WHERE 
            name LIKE {unquoted_name:String}
            AND individual = {individual:bool}
        ORDER BY score
        LIMIT 1
    """
    
    params = {
        "original_name": name.upper(),
        "unquoted_name": f"%{unquote_name(name).upper()}%", 
        "address": address,
        "individual": individual,
    }     
    
    res = client.query(stmt, parameters=params)

    if len(res.result_rows) == 0:
        return []
    else:
        return [
            Person(name=row[0], tax_number=row[1])
            for row in res.result_rows
        ]

In [23]:
(
    unquote_name('Закрытое акционерное общество "Кыштымский медеэлектролитный завод"'),
    unquote_name("Государственное унитарное предприятие Издательство\"Советская Кубань\"")
)

('Кыштымский медеэлектролитный завод', 'Советская Кубань')

In [24]:
search_by_like_name_match(
    'Закрытое акционерное общество "Кыштымский медеэлектролитный завод"',
    '456870, Челябинская обл., г. Кыштым, ул. П.Коммуны, 2 Плеханову И.Д',
    individual=False
)

[Person(name='АКЦИОНЕРНОЕ ОБЩЕСТВО "КЫШТЫМСКИЙ МЕДЕЭЛЕКТРОЛИТНЫЙ ЗАВОД"', tax_number='7413000630')]

In [25]:
search_by_like_name_match("Нижегородский научно-исследовательский институт радиотехники", "", individual=False)

[Person(name='ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ "НИЖЕГОРОДСКИЙ НАУЧНО-ИССЛЕДОВАТЕЛЬСКИЙ ИНСТИТУТ РАДИОТЕХНИКИ"', tax_number='5261000043')]

In [26]:
search_by_like_name_match(
    'Государственное бюджетное учреждение Свердловской области "Уральский научно-исследовательский институт дерматовенерологии и иммунопатологии" (ГБУ СО "УрНИИДВиИ")',
    '',
    individual=False
)

[]

In [27]:
search_by_like_name_match('Производственное республиканское унитарное предприятие "Завод полупроводниковых приборов"', address=None, individual=False)

[Person(name='ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ "ЗАВОД ПОЛУПРОВОДНИКОВЫХ ПРИБОРОВ"', tax_number='1200001518')]

In [28]:
def search_by_tokens_match(name: Optional[str], address: Optional[str], individual: bool) -> Optional[Person]:
    if name is None or name == "":
        return []
    
    if address is None or address == "":
        address = "фыва" # meaningless, score ~ 1 (least similar) for ngramDistance
        
    tokens = [
        token.upper()
        for token in nltk.word_tokenize(name)
        if token not in stopwords and len(token) > 3
    ]
    
    if len(tokens) < 3:
        return [] # method is slow, so we use it for long names only
    
    stmt = """
        SELECT 
            name,
            tax_number,
            ngramDistance(name, {original_name:String}) as dn,
            ngramDistance(legal_address, {address:String}) as dal,
            ngramDistance(fact_address, {address:String}) as daf,
            length(multiMatchAllIndices(name, {tokens:Array(String)})) / length({tokens:Array(String)}) as ts,
            dn * dal * daf / ts as score
        FROM search.search_base
        WHERE
            ts > 0.5
            AND individual = {individual:bool}
        ORDER BY score
        LIMIT 1         
    """
    
    params = {
        "original_name": name.upper(),
        "tokens": tokens, 
        "address": address,
        "individual": individual,
    }
    
    res = client.query(stmt, parameters=params)

    if len(res.result_rows) == 0:
        return []
    else:
        return [
            Person(name=row[0], tax_number=row[1])
            for row in res.result_rows
        ]

In [29]:
search_by_tokens_match(
    'Государственное бюджетное учреждение Свердловской области "Уральский научно-исследовательский институт дерматовенерологии и иммунопатологии" (ГБУ СО "УрНИИДВиИ")',
    None,
    individual=False
)

[Person(name='ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ СВЕРДЛОВСКОЙ ОБЛАСТИ "УРАЛЬСКИЙ НАУЧНО-ИССЛЕДОВАТЕЛЬСКИЙ ИНСТИТУТ ДЕРМАТОВЕНЕРОЛОГИИ И ИММУНОПАТОЛОГИИ"', tax_number='6664033967')]

In [30]:
search_by_tokens_match('Институт сильноточной электроники СО РАН', None, False)

[Person(name='ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ НАУКИ ИНСТИТУТ СИЛЬНОТОЧНОЙ ЭЛЕКТРОНИКИ СИБИРСКОГО ОТДЕЛЕНИЯ РОССИЙСКОЙ АКАДЕМИИ НАУК', tax_number='7021001375')]

In [35]:
def search(patent: Patent) -> List[Person]:
    result = []
    
    for owner in patent.owners:
        not_found = Person(owner.name, None)
        if owner.country_code and owner.country_code != "RU":
            result.append(not_found)
            continue
            
        if owner.individual is True:
            methods = (
                search_by_exact_name_match,
            )
        else:
            methods = (
                search_by_exact_name_match,
                search_by_like_name_match,
                search_by_tokens_match,
            )
        
        for method in methods:            
            found = method(owner.name, patent.address, owner.individual)
            
            if len(found) == 0:
                continue
            else:
                result.append(found[0])
                break
        else:
            result.append(not_found)
    
    return result

In [4]:
inv_sample = pd.read_csv("../data/opendata/samples/inventions_sample.csv", index_col=0)
mod_sample = pd.read_csv("../data/opendata/samples/models_sample.csv")
des_sample = pd.read_csv("../data/opendata/samples/designs_sample.csv")

In [5]:
inv_sample.head()

,registration number,registration date,application number,application date,authors,authors in latin,patent holders,patent holders in latin,correspondence address,correspondence address in latin,...,application publish number,patent grant publish date,patent grant publish number,revoked patent number,information about the obligation to conclude contract of alienation,expiration date,invention formula numbers for which patent term is prolonged,additional patent,actual,publication URL
133987,2137261,19990910.0,9.811426e+07,19980804.0,Демидов Юрий Михайлович,NaN,Демидов Юрий \n\nМихайлович,NaN,"127560, Москва, ул.Коненкова, 5, кв.16, Демидо...","127560, Moskva, ul.Konenkova, 5, kv.16, Demido...",...,NaN,19990910.0,25.0,NaN,NaN,NaN,NaN,False,False,http://www1.fips.ru/fips_servl/fips_servlet?DB...
626621,2631279,20170920.0,2.016110e+09,20160318.0,Кочетов Олег Савельевич (RU),NaN,Кочетов Олег Савельевич (RU),NaN,"141191, Московская обл., г. Фрязино, ул. Горьк...",NaN,...,NaN,20170920.0,26.0,NaN,На основании пункта 1 статьи 1366 части четвер...,NaN,NaN,False,True,http://www1.fips.ru/fips_servl/fips_servlet?DB...
727305,2731963,20200909.0,2.020112e+09,20200319.0,Хамнагадаев Игорь Алексеевич (RU)\r\nТарбаева ...,\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,NaN,"119991, Москва, ГСП-1, ул. Ломоносовский просп...",NaN,...,NaN,20200909.0,25.0,NaN,NaN,NaN,NaN,False,True,http://www1.fips.ru/fips_servl/fips_servlet?DB...
353856,2357844,20090610.0,2.007130e+09,20070801.0,Афонькин Михаил Григорьевич (RU)\r\nЗвягин Вла...,\r\n\r\n\r\n,Федеральное государственное бюджетное образова...,NaN,NaN,NaN,...,NaN,20090610.0,16.0,NaN,NaN,NaN,NaN,False,False,http://www1.fips.ru/fips_servl/fips_servlet?DB...
189971,2193265,20021120.0,2.001104e+09,20010212.0,Бульбин Юрий Васильевич\r\nБуянов Юрий Иннокен...,\r\n\r\n\r\n\r\n,Сибирский физико-технический институт при Томс...,\r\n,NaN,NaN,...,NaN,20021120.0,32.0,NaN,NaN,NaN,NaN,False,False,http://www1.fips.ru/fips_servl/fips_servlet?DB...


In [37]:
s = des_sample.iloc[0, ]
s

Unnamed: 0                                                                                                                    41814
registration number                                                                                                           81851
registration date                                                                                                        20120516.0
application number                                                                                                     2011502074.0
application date                                                                                                         20110707.0
authors                                                                                                         БЕЛИНГ Кристоф (GB)
authors in latin                                                                                                                NaN
patent holders                                                              

In [46]:
import datetime
reg_date = datetime.datetime.strptime("201601231", "%Y%m%d").date()
reg_date

ValueError: unconverted data remains: 1

In [16]:
country_code_regex = re.compile("\((?a:\w{2})\)")

In [36]:
from collections import Counter
country_codes = country_code_regex.findall("(RU)")
cnt = Counter(country_codes)
cnt.most_common(1)[0][0][1:-1]

'RU'

In [32]:
def test_sample(df):
    result = []
    for _, row in tqdm.tqdm(df.iterrows()):
        patent = parse(row)
        persons = search(patent)
        for owner, person in zip(patent.owners, persons):
            result.append(
                (
                    patent.number, patent.address, person.name, person.tax_number,
                    owner.name, owner.individual, owner.country_code
                )
            )
    
    return pd.DataFrame(
        result, 
        columns=[
            "patent_number", "cor_address", "name", "tax_number", 
            "name_from_patent", "individual", "country"
        ]
    )

In [30]:
inv_test_result = test_sample(inv_sample)
inv_test_result.head()

1000it [06:08,  2.72it/s]


,patent_number,cor_address,name,tax_number,name_from_patent,individual,country
0,2137261,"127560, Москва, ул.Коненкова, 5, кв.16, Демидо...",ДЕМИДОВ ЮРИЙ МИХАЙЛОВИЧ,772637744816,Демидов Юрий Михайлович,True,None
1,2631279,"141191, Московская обл., г. Фрязино, ул. Горьк...",Кочетов Олег Савельевич,None,Кочетов Олег Савельевич,True,RU
2,2731963,"119991, Москва, ГСП-1, ул. Ломоносовский просп...",ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,7728016351,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,False,RU
3,2357844,,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВА...,7706025867,Федеральное государственное бюджетное образова...,False,None
4,2193265,,СИБИРСКИЙ БОТАНИЧЕСКИЙ САД ПРИ ТОМСКОМ ГОСУДАР...,7018024710,Сибирский физико-технический институт при Томс...,False,None


In [247]:
inv_test_result["has_tn"] = inv_test_result["tax_number"].notna()
inv_test_result.groupby(["individual", "has_tn"]).size()

individual  has_tn
False       False     331
            True      449
True        False     391
            True      174
dtype: int64

In [88]:
inv_test_result["has_tn"] = inv_test_result["tax_number"].notna()
inv_test_result.loc[inv_test_result["country"].isna() | (inv_test_result["country"] == "RU")].groupby(["individual", "has_tn"]).size()

individual  has_tn
False       False     121
            True      433
True        False     243
            True      176
dtype: int64

In [161]:
inv_test_result["has_tn"] = inv_test_result["tax_number"].notna()
inv_test_result.loc[
    inv_test_result["country"].isna() | (inv_test_result["country"] == "RU")
].groupby(["individual", "has_tn"]).size().reset_index().assign(share = )

individual  has_tn
False       False       9
            True      490
True        False     243
            True      176
dtype: int64

In [166]:
(490 + 176) / (490 + 176 + 9 + 243) # matched share

0.7254901960784313

In [162]:
inv_test_result.loc[
    (~inv_test_result["has_tn"] & ~inv_test_result["individual"]),
    ["name_from_patent", "cor_address"]
]["name_from_patent"].to_list()

['Российская Федерация, от имени которой выступает Министерство обороны Российской Федерации (Минобороны России)',
 'Акционерное общество "Астрата"',
 'ДАЛЯНЬ СЕЙФ ТЕХНОЛОДЖИ КО., ЛТД',
 'Производственное республиканское унитарное предприятие "Завод полупроводниковых приборов"',
 'БАЙОВЕЙЛ ЛЭБОРЕТЕРИЗ ИНТЕРНЕШНЛ (БАРБАДОС) СРЛ',
 'КНОРР-БРЕМЗЕ ЗЮСТЕМЕ ФЮР НУТЦФАРЦОЙГЕ ГМБХ',
 'ТОРОТРАК (ДИВЕЛОПМЕНТ) ЛИМИТЕД',
 'РОЙЯЛ ЭППЛИАНС МФГ. КО,',
 'ВИСКОФАН, ИНДУСТРИЯ НАВАРРА ДЕ ЭНВОЛЬТУРАС КЕЛЮЛОЗИКАС, С.А.',
 'Тетра-Лаваль Холдингз энд Файнэнс С.А.',
 'ГЕОРГ ФИШЕР ВАГА Н.В.',
 'ФИШЕР КОНТРОЛЗ ИНТЕРНЭШНЛ ЛЛС',
 'ЭРСЕЛЬ',
 'ЛТС ЛОМАНН ТЕРАПИ-СИСТЕМ АГ',
 'КНОРР-БРЕМЗЕ ЗЮСТЕМЕ ФЮР НУТЦФАРЦОЙГЕ ГМБХ',
 'ОЙ ЭКСПАНСИО ЭНЖИНИРИНГ ЛИМИТЕД',
 'растений им. К.А. Тимирязева',
 'ЭЛ ДЖИ ЭЛЕКТРОНИКС ИНК.',
 'М.Э.П. МАККИНЕ ЭЛЕТТРОНИКЕ ПЬЕГАТРИЧИ С.П.А.',
 'Майкрософт Текнолоджи Лайсенсинг, ЭлЭлСи',
 'Коммонвелт Сайентифик энд Индастриал Рисерч Организейшн',
 '',
 'Хайперион Каталайзис Интернэшнл Инк.',
 'Са

In [175]:
inv_test_result.to_excel("inv_test_result_v2.xlsx")

In [165]:
mod_test_result = test_sample(mod_sample)
mod_test_result.head()

1000it [12:34,  1.33it/s]


,patent_number,cor_address,name,tax_number,name_from_patent,individual,country
0,120365,"660133, г.Красноярск, ул. Авиаторов, 1, стр.1,...","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АРНИКА""",2460018787,"Общество с ограниченной ответственностью ""Арника""",False,RU
1,32373,"198328, Санкт-Петербург, пр-т маршала Захарова...",БЕЛЯЕВ АЛЕКСАНДР ГЕННАДЬЕВИЧ,780708385189,Беляев Александр Геннадьевич,True,RU
2,96928,"117405, Москва, Варшавское ш., 143, корп.1, кв...","АКЦИОНЕРНОЕ ОБЩЕСТВО ""СВЯЗЬ ИНЖИНИРИНГ М""",7713551934,"Закрытое акционерное общество ""Связь инжинирин...",False,RU
3,116992,"141103, Московская обл., г. Щелково-3, ул. Гаг...",Филиппов Валерьян Степанович,None,Филиппов Валерьян Степанович,True,RU
4,169346,"680031, г. Хабаровск, ул. Карла Маркса, 144а, ...",БЕЗМАТЕРНЫХ РОМАН ВИКТОРОВИЧ,272511823375,Безматерных Роман Викторович,True,RU


In [254]:
mod_test_result["has_tn"] = mod_test_result["tax_number"].notna()
mod_test_result.groupby(["individual", "has_tn"]).size()

individual  has_tn
False       False     155
            True      563
True        False     225
            True      283
dtype: int64

In [167]:
mod_test_result["has_tn"] = mod_test_result["tax_number"].notna()
mod_test_result.loc[mod_test_result["country"].isna() | (mod_test_result["country"] == "RU")].groupby(["individual", "has_tn"]).size()

individual  has_tn
False       False      15
            True      682
True        False     182
            True      283
dtype: int64

In [170]:
(682 + 283) / (682 + 283 + 15 + 182) # matched share

0.8304647160068847

In [174]:
mod_test_result.to_excel("mod_test_result_v2.xlsx")

In [168]:
des_test_result = test_sample(des_sample)
des_test_result.head()

1000it [05:38,  2.95it/s]


,patent_number,cor_address,name,tax_number,name_from_patent,individual,country
0,81851,"191186, Санкт-Петербург, а/я 230, АРС-ПАТЕНТ, ...",Геберит Интернешенл АГ,None,Геберит Интернешенл АГ,True,CH
1,110832,"142455, \nМосковская обл., \nНогинский р-н, \n...","П.П.Х. ""АДАМЕКС"" Я.Каронь, Э.Каспшык, А. Каспш...",None,"П.П.Х. ""АДАМЕКС"" Я.Каронь, Э.Каспшык, А. Каспш...",False,PL
2,139296,"124460, \nМосква, \nг. Зеленоград, \nа/я 200,\...","АКЦИОНЕРНОЕ ОБЩЕСТВО ""КАМА""",1650404549,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""КАМА""",False,RU
3,94890,"660111, г.Красноярск, ул. Пограничников, 37, с...","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ОБЪЕ...",3804039638,"Общество с ограниченной ответственностью ""Объе...",False,RU
4,125266,"117042, \nМосква, \nПлавский проезд, д. 1, кв....","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИНДИ...",7840034022,Индивидуальный предприниматель Бирюков Денис В...,False,RU


In [257]:
des_test_result["has_tn"] = des_test_result["tax_number"].notna()
des_test_result.groupby(["individual", "has_tn"]).size()

individual  has_tn
False       False     300
            True      343
True        False     276
            True      148
dtype: int64

In [169]:
des_test_result["has_tn"] = des_test_result["tax_number"].notna()
des_test_result.loc[des_test_result["country"].isna() | (des_test_result["country"] == "RU")].groupby(["individual", "has_tn"]).size()

individual  has_tn
False       False      15
            True      383
True        False      54
            True      147
dtype: int64

In [171]:
(383 + 147) / (383 + 147 + 15 + 54) # matched share

0.8848080133555927

In [173]:
des_test_result.to_excel("des_test_result_v2.xlsx")

In [188]:
inv_sample.columns

Index(['Unnamed: 0', 'registration number', 'registration date',
       'application number', 'application date', 'authors', 'authors in latin',
       'patent holders', 'patent holders in latin', 'correspondence address',
       'correspondence address in latin', 'invention name',
       'patent starting date',
       'Crimean invention application number for state registration in Ukraine',
       'Crimean invention application date for state registration in Ukraine',
       'Crimean invention patent number in Ukraine',
       'receipt date of additional data to application',
       'date of application to which additional data has been received',
       'number of application to which additional data has been received',
       'initial application number', 'initial application date',
       'initial application priority date', 'previous application number',
       'previous application date', 'paris convention priority number',
       'paris convention priority date',
       'paris c

In [4]:
c = pd.read_csv("../data/opendata/inventions.csv", chunksize=10)

In [5]:
c.get_chunk(1).columns

Index(['registration number', 'registration date', 'application number',
       'application date', 'authors', 'authors in latin', 'patent holders',
       'patent holders in latin', 'correspondence address',
       'correspondence address in latin', 'invention name',
       'patent starting date',
       'Crimean invention application number for state registration in Ukraine',
       'Crimean invention application date for state registration in Ukraine',
       'Crimean invention patent number in Ukraine',
       'receipt date of additional data to application',
       'date of application to which additional data has been received',
       'number of application to which additional data has been received',
       'initial application number', 'initial application date',
       'initial application priority date', 'previous application number',
       'previous application date', 'paris convention priority number',
       'paris convention priority date',
       'paris convention prio

In [52]:
for chunk in c:
    print(chunk)
    break

    registration number  registration date  application number  \
1                 84605                NaN                 NaN   
2                113967         20120310.0        5.711310e+05   
3                120650         20120927.0        2.012127e+09   
4                206015                NaN                 NaN   
5                239926                NaN        1.041801e+06   
6                601043         19780405.0        2.099415e+06   
7                778776         19801115.0        2.391796e+06   
8               1002282         19830307.0        3.264511e+06   
9               1267657         19951010.0        3.823411e+06   
10              1296557         19870315.0        3.838883e+06   

    application date                                            authors  \
1                NaN  Руденко Сергей Михайлович (RU)\r\nМакедонский ...   
2         19570411.0  Емельянов Роман Александрович (RU)\r\nКузьменк...   
3         20120702.0  Антонов Игорь Николаевич (

In [38]:
def process_file(in_file: str, out_file: str, kind: int, chunksize: int = 1e2):
    input_df = pd.read_csv(in_file, chunksize=chunksize)
    
    for chunk in tqdm.tqdm(input_df):
        chunk_result = []
        for _, row in chunk.iterrows():
            try:
                patent = parse(row)
                persons = search(patent)
            except Exception as e:
                print(e)
                continue
            
            rows = [
                (patent.number, person.name, person.tax_number)
                for person in persons
            ]
            chunk_result.extend(rows)
        
        result_df = pd.DataFrame(
            chunk_result, 
            columns=["patent_number", "person_name", "person_tax_number"]
        )
        result_df["patent_kind"] = kind
        
        if not pathlib.Path(out_file).exists():
            result_df.to_csv(out_file, mode="w", index=False)
        else:
            result_df.to_csv(out_file, mode="a", index=False, header=False)

In [39]:
process_file("../data/opendata/models.csv", "models_match.csv", 2)

4it [03:08, 47.03s/it]


KeyboardInterrupt: 